In [2]:
import tensorflow as tf

## 1. Enqueuing & Dequeuing single data

In [5]:
q = tf.FIFOQueue(capacity=10, dtypes=[tf.float32], shapes=[[2]], name="q", shared_name="shared_q")
training_instance = tf.placeholder(tf.float32, shape=(2))
enqueue = q.enqueue([training_instance])
dequeue = q.dequeue()

with tf.Session() as sess:
    sess.run(enqueue, feed_dict={training_instance: [1., 2.]})
    sess.run(enqueue, feed_dict={training_instance: [3., 4.]})
    sess.run(enqueue, feed_dict={training_instance: [5., 6.]})
    print(sess.run(dequeue))
    print(sess.run(dequeue))
    print(sess.run(dequeue))

[ 1.  2.]
[ 3.  4.]
[ 5.  6.]


In [3]:
q

## 2. Enqueuing & Dequeuing many data

In [2]:
q_many = tf.FIFOQueue(capacity=10, dtypes=[tf.float32], shapes=[[2]], name="q_many", shared_name="shared_q_many")
training_instance_many = tf.placeholder(tf.float32, shape=(None, 2))
enqueue_many = q_many.enqueue_many([training_instance_many])
batch_size = 2
dequeue_mini_batch = q_many.dequeue_many(batch_size)

with tf.Session() as sess:
    sess.run(enqueue_many, feed_dict={training_instance_many: [[11., 22.], [33., 44.], [55., 66.]]})
    print(sess.run(dequeue_mini_batch))

[[ 11.  22.]
 [ 33.  44.]]


## 3. Queues of tuples

In [10]:
q_tuple = tf.FIFOQueue(capacity=10, dtypes=[tf.int32, tf.float32], shapes=[[], [3, 2]], name="q_tuple", 
                       shared_name="shared_q_tuple")
a = tf.placeholder(tf.int32, shape=())
b = tf.placeholder(tf.float32, shape=(3, 2))
enqueue = q_tuple.enqueue((a, b))
dequeue_a, dequeue_b = q_tuple.dequeue()
batch_size = 2
dequeue_as, dequeue_bs = q_tuple.dequeue_many(batch_size)

with tf.Session() as sess:
    sess.run(enqueue, feed_dict={a: 10, b: [[1., 2.], [3., 4.], [5., 6.]]})
    sess.run(enqueue, feed_dict={a: 11, b: [[7., 2.], [8., 4.], [9., 6.]]})
    sess.run(enqueue, feed_dict={a: 12, b: [[3., 2.], [6., 4.], [1., 6.]]})
    a_val, b_val = sess.run([dequeue_a, dequeue_b])
    print(a_val)
    print(b_val)
    a, b = sess.run([dequeue_as, dequeue_bs])
    print(a)
    print(b)

10
[[ 1.  2.]
 [ 3.  4.]
 [ 5.  6.]]
[11 12]
[[[ 7.  2.]
  [ 8.  4.]
  [ 9.  6.]]

 [[ 3.  2.]
  [ 6.  4.]
  [ 1.  6.]]]


In [4]:
devices = sess.list_devices()
for d in devices:
  print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0


## 4. RandomShuffleQueue

In [3]:
q = tf.RandomShuffleQueue(capacity=30, min_after_dequeue=10, dtypes=[tf.float32], shapes=[()],
                          name="q_random", shared_name="shared_q_random")
training_instance = tf.placeholder(tf.float32, shape=(None, ))
enqueue = q.enqueue_many([training_instance])
dequeue = q.dequeue_many(3)

with tf.Session() as sess:
    sess.run(enqueue, feed_dict={training_instance: [1., 2., 3., 4., 5.]})
    sess.run(enqueue, feed_dict={training_instance: [11., 2., 3., 4., 5.]})
    sess.run(enqueue, feed_dict={training_instance: [111., 2., 3., 4., 5.]})
    sess.run(enqueue, feed_dict={training_instance: [11111., 2., 3., 4., 5.]})
    sess.run(enqueue, feed_dict={training_instance: [123., 2., 3., 4., 5.]})
    print(sess.run(dequeue))
    print(sess.run(dequeue))
    # if we set dequeue batch size to 6, then the last can't dequeue, because min_after_dequeue=10
    print(sess.run(dequeue)) 

[ 3.  4.  3.]
[  3.00000000e+00   4.00000000e+00   1.11110000e+04]
[ 2.  4.  2.]


## 5. PaddingFifiQueue

In [5]:
q_padding = tf.PaddingFIFOQueue(capacity=50, dtypes=[tf.float32], shapes=[(None, None)], 
                                name="q_padding", shared_name="shared_q_padding")
v = tf.placeholder(tf.float32, shape=(None, None))
enqueue = q_padding.enqueue([v])
dequeue = q_padding.dequeue_many(3)

with tf.Session() as sess:
    sess.run(enqueue, feed_dict={v: [[1., 2.], [3., 4.], [5., 6.]]})        # 3x2
    sess.run(enqueue, feed_dict={v: [[1.]]})                                # 1x1
    sess.run(enqueue, feed_dict={v: [[7., 8., 9., 5.], [6., 7., 8., 9.]]})  # 2x4
    print(sess.run(dequeue))

[[[ 1.  2.  0.  0.]
  [ 3.  4.  0.  0.]
  [ 5.  6.  0.  0.]]

 [[ 1.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 7.  8.  9.  5.]
  [ 6.  7.  8.  9.]
  [ 0.  0.  0.  0.]]]
